In [48]:
import os
import pandas as pd
import cv2
import numpy as np
import math
import xml.dom.minidom
import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Polygon
from PIL import Image
from glob import glob

## Functions 

In [49]:
# Create directories

def create_dir(dirname):
    
    try:
        os.mkdir(dirname)
    except OSError:
        print ("Creation of the directory %s failed" % dirname)
    else:
        print ("Successfully created the directory %s " % dirname)

## Create directories and path to dhSegment output

In [50]:
# Directories and create folders
root_dir = "../Outputs/MM_seg/"

create_dir(root_dir)

# Choose the name of the folder to store all the results and resulting mark database
folder = "753_1588848"
reg_dir = "../Outputs/MM_seg/" + folder + "/model_output_regions/"
DB_dir = "../Outputs/MM_seg/" + folder + "/DB/"
xml_dir = "processed_marks/page_xml/*"
xmls = glob(xml_dir)

create_dir(DB_dir)
create_dir(reg_dir)

Creation of the directory ../Outputs/MM_seg/ failed
Creation of the directory ../Outputs/MM_seg/753_1588848/DB/ failed
Creation of the directory ../Outputs/MM_seg/753_1588848/model_output_regions/ failed


In [51]:
# Check the files in xml directory

for page in xmls:
    print(page)
    
print(len(xmls))

processed_marks/page_xml/113_21_0057.xml
processed_marks/page_xml/114_7_0076.xml
processed_marks/page_xml/105_11_0076.xml
processed_marks/page_xml/114_7_0078.xml
processed_marks/page_xml/105_6_0038.xml
processed_marks/page_xml/105_11_0105.xml
processed_marks/page_xml/105_11_0117.xml
processed_marks/page_xml/94_20_0082.xml
processed_marks/page_xml/105_11_0077.xml
processed_marks/page_xml/114_7_0077.xml
processed_marks/page_xml/105_11_0103.xml
processed_marks/page_xml/94_20_0059.xml
processed_marks/page_xml/105_11_0099.xml
13


## Extract XML regions

In [52]:
# Extract model output data from XMLs (XML has the coordinates as x1,y1 x2,y2 x3,y3 x4,y4)
                                                               # (bl, br, tr, tl)

output = []

for page in xmls:

    data = []
    regions = []    
    doc = xml.dom.minidom.parse(page)
    
    name = doc.getElementsByTagName("Page")
    print("\nPage being analysed is " + name[0].getAttribute("imageFilename"))
    data.append(name[0].getAttribute("imageFilename")[:-4])
    
    Nodelist = doc.getElementsByTagName('Coords')
    
    for node in Nodelist:
        coords = []
        coordsTemp = node.getAttribute("points")
        coordsTemp = coordsTemp.split(' ')
        for x in coordsTemp:
            temp = x.split(',')
            coords.append((int(temp[0]),int(temp[1])))

        regions.append(coords)
        
    data.append(regions)
    output.append(data)

print("")
print(len(output))


Page being analysed is 113_21_0057.png

Page being analysed is 114_7_0076.png

Page being analysed is 105_11_0076.png

Page being analysed is 114_7_0078.png

Page being analysed is 105_6_0038.jpg

Page being analysed is 105_11_0105.png

Page being analysed is 105_11_0117.png

Page being analysed is 94_20_0082.jpg

Page being analysed is 105_11_0077.png

Page being analysed is 114_7_0077.png

Page being analysed is 105_11_0103.png

Page being analysed is 94_20_0059.jpg

Page being analysed is 105_11_0099.png

13


## Pixel-based heuristic, create projection signatures, analysis and mark images creation

In [53]:
# Create lists to eventually produce a CSV
# Original prize paper file
fnames = []
# Name of mark image
mark_ids = []
# Coordinates
co = []
# Counter for the mark instance
m_count = 0

# Parameters for the user to choose (The average size of a merchant's size)
av_mark_size = 250


for x in output:
    
    # Load the original prize paper image
    filename = x[0]
    regions = x[1]
    file_dir = "../SampleSetsPNG/" + filename + ".png"
    
    print("\nOpen " + filename)
    img = cv2.imread(file_dir)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(img,100,200)
    
    for region in regions:
        # Get the cropped image
        crop = img[region[1][1]:region[2][1], region[0][0]:region[1][0]]
        ecrop = edges[region[1][1]:region[2][1], region[0][0]:region[1][0]]
        
        # Check the region size, if it is a single MM or not.
        if ecrop.shape[0] < av_mark_size:
            # Add to DB and dataframe lists
            cv2.imwrite(DB_dir + '/mark' + str(m_count) + '.png', crop)
            fnames.append(filename)
            mark_ids.append("mark" + str(m_count))
            co.append(region)
            m_count += 1
            
            continue
        else:
            
            # Create and analyse the projection signature
            esig = np.sum(ecrop, axis=1)/255

            bin_sig = []
            peaks = []
            p = []
            
            #Produce a binary signal, store pixel values for peak and append to peaks.
            for n in range(len(esig)):
                if esig[n] >= 2:
                    bin_sig.append(5)
                    p.append(n)
                    if n == len(esig)-1:
                        peaks.append(p)
                else:
                    bin_sig.append(0)
                    if len(p) != 0:
                        peaks.append(p)
                        p = []
                    else:
                        continue
            
            ubin_sig = list(bin_sig)
                
            # Filter signal (Remove unlikely peaks)
            indicies = []
            for x in range(len(peaks)):
                peak = peaks[x]
                if len(peak) < 50:
                    for y in range(len(peak)):
                        bin_sig[peak[y]] = 0
                    indicies.append(x)
                else:
                    continue
            
            # Pop the false peaks from the largest index first
            indicies.reverse()
            for x in indicies:
                peaks.pop(x)
            
            # Check the number of peaks
            if len(peaks) == 1:
                # Add to DB and dataframe lists
                cv2.imwrite(DB_dir + '/mark' + str(m_count) + '.png', crop)
                fnames.append(filename)
                mark_ids.append("mark" + str(m_count))
                co.append(region)
                m_count += 1
                
                continue
            else:
                
                # Extract the newly segmented merchant's marks
                
                #plt.plot(esig, label="Projection signature")
                #plt.plot(ubin_sig, label="Unfiltered binary signal", linestyle="dotted")
                #plt.plot(bin_sig, label="Binary signal", linestyle="dashed")
                #plt.xlabel("Row")
                #plt.ylabel("No. of edge pixels")
                #plt.legend()
                #plt.title("No. of edge pixels vs row")

                #plt.savefig(reg_dir + filename + '/mark_seg_signal/region' + str(m_count) + '.svg')
                #plt.show()

                m = 0
                for peak in peaks:
                    top = region[1][1] + peak[0] - 10
                    bottom = region[1][1] + peak[len(peak)-1] + 10
                    left = region[0][0] - 20
                    right = region[1][0] + 20
                    ms_crop = img[top:bottom, region[0][0]:region[1][0]]
                    cv2.imwrite(DB_dir + '/mark' + str(m_count) + '.png', ms_crop)
                    m += 1
                    
                    # Add to DB and dataframe lists
                    tl = (left,top)
                    tr = (right,top)
                    br = (right,bottom)
                    bl = (left,bottom)
                    fnames.append(filename)
                    mark_ids.append("mark" + str(m_count))
                    co.append([tl,tr,br,bl])
                    m_count += 1
                    


Open 113_21_0057

Open 114_7_0076

Open 105_11_0076

Open 114_7_0078

Open 105_6_0038

Open 105_11_0105

Open 105_11_0117

Open 94_20_0082

Open 105_11_0077

Open 114_7_0077

Open 105_11_0103

Open 94_20_0059

Open 105_11_0099


## Create the CSV for the mark database

In [54]:
db_dict = {
    "filename": fnames,
    "mark_id": mark_ids,
    "co_ordinates": co
}

print(db_dict)

mark_DB = pd.DataFrame(db_dict)

print(mark_DB)

mark_DB.to_csv(DB_dir + "outputDB.csv", index=True)

{'filename': ['113_21_0057', '113_21_0057', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '114_7_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '105_11_0076', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_7_0078', '114_